In [2]:
! pip install builtwith

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for builtwith: filename=builtwith-1.3.4-py3-none-any.whl size=36133 sha256=386f10871930d563f18f6f3a1ed4a009b0ea77254c925eebffbe5cb7beeadbf5
  Stored in directory: c:\users\syafiq azizi\appdata\local\pip\cache\wheels\7f\2d\b2\606e3df914d4aeeab99c4a4e3e9a61673d2293c2e346db00c8
Successfully built builtwith


  DEPRECATION: Building 'builtwith' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'builtwith'. Discussion can be found at https://github.com/pypa/pip/issues/6334

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import builtwith

# Analisis teknologi yang digunakan
res = builtwith.parse('https://pta.trunojoyo.ac.id')
print(res)

{'web-servers': ['Nginx'], 'javascript-frameworks': ['jQuery', 'jQuery UI']}


In [4]:
import requests
from bs4 import BeautifulSoup

def crawl_website(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes

        soup = BeautifulSoup(response.content, 'html.parser')

        # Ambil semua judul h1, h2, h3
        headings = soup.find_all(['h1', 'h2', 'h3'])
        for heading in headings:
            print(f"{heading.name}: {heading.get_text()}")

        # Ambil semua link
        links = soup.find_all('a', href=True)
        for link in links:
            print(f"URL: {link['href']} | Teks: {link.get_text()}")

    except requests.exceptions.RequestException as e:
        print(f"Terjadi kesalahan saat mengakses {url}: {e}")

# Gunakan fungsi
crawl_website("https://pta.trunojoyo.ac.id/")

h2: Daftar Karya Ilmiah
URL: index.html | Teks: 
URL: # | Teks: 14677Journal
URL: https://pta.trunojoyo.ac.id/ | Teks: Beranda
URL: https://pta.trunojoyo.ac.id/c_search/ | Teks: Pencarian
URL: https://pta.trunojoyo.ac.id/c_template/ | Teks: Download
URL: https://library.trunojoyo.ac.id/detil.php?id=23 | Teks: Petunjuk Upload
URL: https://pta.trunojoyo.ac.id/c_contact/ | Teks: Kontak
URL: # | Teks: STRATEGI PENGEMBANGAN MAKANAN DAN MINUMAN KHAS PULAU GILIGENTING GUNA MENDUKUNG PARIWISATA BERKELANJUTAN
URL: https://pta.trunojoyo.ac.id/welcome/detail/170361100003 | Teks: Selengkapnya
URL: # | Teks: PERUMUSAN SANKSI PIDANA BAGI MASYARAKAT SEKITAR HUTAN YANG MELAKUKAN PENCURIAN KAYU MILIK NEGARA DALAM UNDANG-UNDANG NOMOR 18 TAHUN 2013
URL: https://pta.trunojoyo.ac.id/welcome/detail/170111100053 | Teks: Selengkapnya
URL: # | Teks: Peran Teor Motivasi Herzberg Sebagai Mediator Self Efficacy, Lingkungan Kerja Dalam Meningkatkan Prestasi Kerja Pegawai ( Kantor Jasa Penilai Publik Guntur Eki And

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys

def get_text_or_na(soup, selectors):
    """Mencoba beberapa selektor untuk menemukan teks, mengembalikan 'N/A' jika tidak ditemukan."""
    for selector in selectors:
        element = soup.select_one(selector)
        if element:
            text = element.get_text(strip=True)
            if text and text.strip():
                return text.strip()
    return 'N/A'

def get_data_from_span(soup, text_contains):
    """Mencari span yang berisi teks tertentu dan mengekstrak nama setelah titik dua."""
    span = soup.select_one(f'span:-soup-contains("{text_contains}")')
    if span:
        text = span.get_text(strip=True)
        if ':' in text:
            return text.split(':', 1)[1].strip()
    return 'N/A'

def get_abstract_robust(soup, keywords):
    """
    Mencari tag heading (b) yang teksnya mengandung salah satu dari kata kunci
    dan mengekstrak paragraf berikutnya.
    """
    for tag_b in soup.find_all('b'):
        tag_text = tag_b.get_text(strip=True)
        if any(keyword.lower() in tag_text.lower() for keyword in keywords):
            parent_div = tag_b.find_parent('div')
            if parent_div:
                next_div_sibling = parent_div.find_next_sibling('div')
                if next_div_sibling:
                    p_tag = next_div_sibling.find('p', align="justify")
                    if p_tag:
                        abstract_text = p_tag.get_text(strip=True)
                        if abstract_text:
                            return abstract_text
    return 'N/A'

def get_total_pages(soup):
    """
    Mengekstrak total jumlah halaman dari navigasi paginasi.
    """
    try:
        pagination = soup.select_one('ol.pagination')
        if pagination:
            last_page_li = pagination.select('li')[-1]
            last_page_link = last_page_li.select_one('a')
            if last_page_link and 'href' in last_page_link.attrs:
                url_path = last_page_link['href']
                return int(url_path.split('/')[-1])
    except (IndexError, ValueError, KeyError):
        pass
    return 1

def scrape_pta_limited():
    """
    Scrapes a limited number of titles and abstracts (max 100 per program)
    from the Universitas Trunojoyo Madura's final project repository (PTA).
    """
    base_url = "https://pta.trunojoyo.ac.id/"
    MAX_RECORDS_PER_PRODI = 100

    prodi_data = []
    try:
        r = requests.get(base_url)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, "html.parser")
        prodi_links = soup.find_all('a', href=lambda href: href and 'c_search/byprod/' in href)

        for link in prodi_links:
            prodi_name = link.get_text(strip=True)
            prodi_url = link['href']
            prodi_data.append({'name': prodi_name, 'url': prodi_url})

    except requests.exceptions.RequestException as e:
        print(f"Error fetching the main page: {e}", file=sys.stderr)
        return pd.DataFrame()

    all_scraped_data = {
        "penulis": [], "judul": [], "pembimbing_pertama": [], "pembimbing_kedua": [],
        "abstrak_indonesia": [], "abstrak_inggris": [], "prodi": []
    }
    
    total_data_count = 0

    # Print table header for live console output
    print("=" * 150)
    print(f"{'No.':<4} | {'Prodi':<20} | {'Penulis':<25} | {'Judul':<60} | {'Pembimbing I':<25} | {'Pembimbing II':<25}")
    print("=" * 150)

    for prodi in prodi_data:
        print(f"\nScraping data for program: {prodi['name']}")
        print("-" * 150)
        
        scraped_count = 0
        page = 1
        
        while scraped_count < MAX_RECORDS_PER_PRODI:
            url = f"{prodi['url']}/{page}"
            
            try:
                r = requests.get(url)
                r.raise_for_status()
                soup = BeautifulSoup(r.content, "html.parser")
                jurnals = soup.select('li[data-cat="#luxury"]')
                
                if not jurnals:
                    print(f"No more journals found on page {page}. Stopping for this program.")
                    break
                
                for jurnal in jurnals:
                    if scraped_count >= MAX_RECORDS_PER_PRODI:
                        break
                        
                    jurnal_url = jurnal.select_one('a.gray.button')['href']
                    
                    try:
                        response = requests.get(jurnal_url)
                        response.raise_for_status()
                        soup1 = BeautifulSoup(response.content, "html.parser")
                        isi = soup1.select_one('div#content_journal')
                        
                        if isi:
                            judul = get_text_or_na(isi, ['a.title', 'b.title', 'h2.title'])
                            penulis = get_data_from_span(isi, "Penulis")
                            pembimbing_pertama = get_data_from_span(isi, "Dosen Pembimbing I")
                            pembimbing_kedua = get_data_from_span(isi, "Dosen Pembimbing II")
                            abstrak_indonesia = get_abstract_robust(isi, ["Abstraksi", "Abstrak"])
                            abstrak_inggris = get_abstract_robust(isi, ["Abstraction", "Abstract", "ABSTRACT"])

                            all_scraped_data["penulis"].append(penulis)
                            all_scraped_data["judul"].append(judul)
                            all_scraped_data["pembimbing_pertama"].append(pembimbing_pertama)
                            all_scraped_data["pembimbing_kedua"].append(pembimbing_kedua)
                            all_scraped_data["abstrak_indonesia"].append(abstrak_indonesia)
                            all_scraped_data["abstrak_inggris"].append(abstrak_inggris)
                            all_scraped_data["prodi"].append(prodi['name'])
                            
                            total_data_count += 1
                            scraped_count += 1
                            
                            # Print data to console in a formatted table row
                            print(f"{total_data_count:<4} | {prodi['name'][:20]:<20} | {penulis[:25]:<25} | {judul[:60]:<60} | {pembimbing_pertama[:25]:<25} | {pembimbing_kedua[:25]:<25}")
                            
                    except Exception as e:
                        print(f"ERROR: An error occurred while processing URL: {jurnal_url} - {e}", file=sys.stderr)
                
                time.sleep(1) # Added delay to avoid overloading the server
                page += 1
                
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL {url}: {e}", file=sys.stderr)
                break
                
    df = pd.DataFrame(all_scraped_data)
    df.to_csv("pta_limited_data.csv", index=False)
    print("\n---")
    print(f"Scraping finished. Total data scraped: {total_data_count}")
    print("Data saved to pta_limited_data.csv")
    print("---")
    return df

# Run the full scraping process
scrape_pta_limited()

No.  | Prodi                | Penulis                   | Judul                                                        | Pembimbing I              | Pembimbing II            

Scraping data for program: Ilmu Hukum
------------------------------------------------------------------------------------------------------------------------------------------------------
1    | Ilmu Hukum           | Dyah Ayu Citra Seza       | Implementasi Fungsi Legislasi Dewan Perwakilan Rakyat Daerah | Yudi Widagdo Harimurti, S | Safi', SH., MH           
2    | Ilmu Hukum           | Maulina Nurlaily          | Pertanggungjawaban Pidana Direksi BUMN (Persero)
(Anotasi P | Tolib Effendi, SH., MH.   | Dr. Eni Suastuti, SH., Mh
3    | Ilmu Hukum           | Moh. Samsul Hidayat       | Analisis Terhadap Kekosongan Hukum dalam Pengawasan Peredara | Tolib Effendi, SH., MH.   | Agus Ramdlany, SH., MH.  
4    | Ilmu Hukum           | TOMMY ADITYA PARLINDUNGAN | PERLINDUNGAN HUKUM BAGI KONSUMEN ATAS PRODUK ELEKTRON

KeyboardInterrupt: 